Job 1, replicate the forward pass of a single MLP layer from GPTNeoXMLP

In [30]:
from modeling_gpt_neox import GPTNeoXForCausalLM
model = GPTNeoXForCausalLM.from_pretrained("trl-internal-testing/tiny-random-GPTNeoXForCausalLM")
from modeling_gpt_neox import GPTNeoXMLP
mlp = GPTNeoXMLP(model.config)

In [31]:
model.state_dict().keys()


odict_keys(['gpt_neox.embed_in.weight', 'gpt_neox.layers.0.input_layernorm.weight', 'gpt_neox.layers.0.input_layernorm.bias', 'gpt_neox.layers.0.post_attention_layernorm.weight', 'gpt_neox.layers.0.post_attention_layernorm.bias', 'gpt_neox.layers.0.attention.bias', 'gpt_neox.layers.0.attention.masked_bias', 'gpt_neox.layers.0.attention.rotary_emb.inv_freq', 'gpt_neox.layers.0.attention.query_key_value.weight', 'gpt_neox.layers.0.attention.query_key_value.bias', 'gpt_neox.layers.0.attention.dense.weight', 'gpt_neox.layers.0.attention.dense.bias', 'gpt_neox.layers.0.mlp.dense_h_to_4h.weight', 'gpt_neox.layers.0.mlp.dense_h_to_4h.bias', 'gpt_neox.layers.0.mlp.dense_4h_to_h.weight', 'gpt_neox.layers.0.mlp.dense_4h_to_h.bias', 'gpt_neox.layers.1.input_layernorm.weight', 'gpt_neox.layers.1.input_layernorm.bias', 'gpt_neox.layers.1.post_attention_layernorm.weight', 'gpt_neox.layers.1.post_attention_layernorm.bias', 'gpt_neox.layers.1.attention.bias', 'gpt_neox.layers.1.attention.masked_bias',

In [32]:
mlp.state_dict()

OrderedDict([('dense_h_to_4h.weight',
              tensor([[-0.0196, -0.0990,  0.0053,  ..., -0.1003,  0.0830, -0.1078],
                      [-0.1272,  0.0624,  0.0038,  ...,  0.0646, -0.1679,  0.0583],
                      [-0.1134,  0.1146, -0.0012,  ...,  0.0271, -0.1597, -0.0135],
                      ...,
                      [-0.0126,  0.1699,  0.0780,  ..., -0.0070,  0.0816,  0.0627],
                      [-0.1656,  0.0370,  0.0644,  ..., -0.1699,  0.0368,  0.1037],
                      [-0.0942, -0.0225,  0.1011,  ...,  0.0172,  0.1001,  0.0858]])),
             ('dense_h_to_4h.bias',
              tensor([ 0.0530, -0.0849, -0.0423, -0.0756,  0.0913, -0.0701, -0.0461,  0.1709,
                       0.1040, -0.0970,  0.1308, -0.0084,  0.0234, -0.1548,  0.0129, -0.1482,
                      -0.1448, -0.1171, -0.0652,  0.0490, -0.1079, -0.0694, -0.0493, -0.0803,
                      -0.1245,  0.0244, -0.0565,  0.1330,  0.1505,  0.0260, -0.1201,  0.0481,
                

In [33]:
# rename the keys to match the mlp
new_state_dict = {}
for k, v in model.state_dict().items():
    if k.startswith("gpt_neox.layers.1.mlp."):
        new_state_dict[k.replace("gpt_neox.layers.1.mlp.", "")] = v

mlp.load_state_dict(new_state_dict)

<All keys matched successfully>

In [34]:
import torch
inputState = torch.randn(2, 32)
mlp(inputState)

tensor([[-4.8557e-03, -7.1405e-03,  7.3479e-04,  1.7308e-03,  1.0328e-02,
          5.5924e-03,  1.7156e-03,  8.4459e-03,  8.2332e-03,  3.5740e-03,
         -8.4398e-04, -3.7490e-03, -2.2467e-03,  5.3495e-03, -3.3868e-03,
         -5.4625e-04,  3.0698e-03, -2.8900e-03, -4.0124e-05, -7.1420e-03,
         -3.3980e-03, -4.9446e-03, -3.6563e-03,  8.5901e-03, -5.9791e-03,
         -2.1923e-03, -1.7515e-03,  3.2583e-03,  9.8841e-03, -3.3359e-03,
          8.8847e-04, -7.3157e-03],
        [ 7.6398e-03,  5.2383e-03,  9.7130e-04,  6.1106e-03, -6.8673e-03,
          4.9993e-03, -1.7223e-03,  6.2101e-03, -7.9022e-03,  1.0678e-02,
          2.7250e-03, -1.6588e-03,  2.5691e-03,  3.7006e-03, -4.1308e-03,
          2.5991e-03, -3.5686e-03,  2.0885e-03, -4.1439e-03,  8.7240e-03,
         -1.2532e-03,  4.4078e-03,  6.3213e-03,  4.7644e-03, -2.6624e-03,
          4.8762e-03, -6.2065e-03,  3.4692e-03, -8.7608e-03,  2.0760e-03,
          7.4033e-04, -7.3633e-03]], grad_fn=<AddmmBackward0>)

In [58]:
import torch.nn as nn
import math

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.hidden_size, config.intermediate_size, bias=True) # n_embd, 4 * n_embd
        self.c_proj  = nn.Linear(config.intermediate_size, config.hidden_size, bias=True)
        self.dropout = nn.Dropout(0) # no dropout
        self.act = torch.nn.GELU()

    def forward(self, x):
        x = self.c_fc(x)
        x = self.act(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

mlp2 = MLP(model.config)
new_state_dict = {
    "c_fc.weight": model.state_dict()["gpt_neox.layers.1.mlp.dense_h_to_4h.weight"],
    "c_fc.bias": model.state_dict()["gpt_neox.layers.1.mlp.dense_h_to_4h.bias"],
    "c_proj.weight": model.state_dict()["gpt_neox.layers.1.mlp.dense_4h_to_h.weight"],
    "c_proj.bias": model.state_dict()["gpt_neox.layers.1.mlp.dense_4h_to_h.bias"],
}
mlp2.load_state_dict(new_state_dict)


<All keys matched successfully>

In [59]:
mlp2(inputState), mlp(inputState)


(tensor([[-4.8557e-03, -7.1405e-03,  7.3479e-04,  1.7308e-03,  1.0328e-02,
           5.5924e-03,  1.7156e-03,  8.4459e-03,  8.2332e-03,  3.5740e-03,
          -8.4398e-04, -3.7490e-03, -2.2467e-03,  5.3495e-03, -3.3868e-03,
          -5.4625e-04,  3.0698e-03, -2.8900e-03, -4.0124e-05, -7.1420e-03,
          -3.3980e-03, -4.9446e-03, -3.6563e-03,  8.5901e-03, -5.9791e-03,
          -2.1923e-03, -1.7515e-03,  3.2583e-03,  9.8841e-03, -3.3359e-03,
           8.8847e-04, -7.3157e-03],
         [ 7.6398e-03,  5.2383e-03,  9.7130e-04,  6.1106e-03, -6.8673e-03,
           4.9993e-03, -1.7223e-03,  6.2101e-03, -7.9022e-03,  1.0678e-02,
           2.7250e-03, -1.6588e-03,  2.5691e-03,  3.7006e-03, -4.1308e-03,
           2.5991e-03, -3.5686e-03,  2.0885e-03, -4.1439e-03,  8.7240e-03,
          -1.2532e-03,  4.4078e-03,  6.3213e-03,  4.7644e-03, -2.6624e-03,
           4.8762e-03, -6.2065e-03,  3.4692e-03, -8.7608e-03,  2.0760e-03,
           7.4033e-04, -7.3633e-03]], grad_fn=<AddmmBackward0>)

In [62]:

# check that the two mlp's are the same
torch.allclose(mlp2(inputState), mlp(inputState))

True